In [2]:
# ============================================================================
# SETUP - Run this cell first!
# ============================================================================

import sys
import os
import json
import glob
import random
from datetime import datetime
from pathlib import Path

import numpy as np

# Jupyter async compatibility
import nest_asyncio
nest_asyncio.apply()

# Try uvloop for performance
try:
    import uvloop
    uvloop.install()
    print("✓ uvloop enabled")
except ImportError:
    print("ℹ uvloop not available")

# Add src to path
sys.path.insert(0, str(Path.cwd()))

# Import our solver
from src.run import solve_task, evaluate_task, run_tasks
from src.models import ArcEvaluator
from src.config import MAX_WORKERS, SOLVER_MODELS

print("✓ Imports loaded")
print(f"✓ Max workers: {MAX_WORKERS}")
print(f"✓ Solver models: {[m['id'] for m in SOLVER_MODELS]}")


✓ uvloop enabled
✓ Imports loaded
✓ Max workers: 100
✓ Solver models: ['gpt-5.2', 'gemini-pro', 'gemini-flash']


In [3]:
# ============================================================================
# RUN N RANDOM TASKS FROM EVALUATION
# ============================================================================

import asyncio

# 👇 CONFIG
NUM_TASKS = 30  # Number of random tasks to run
EVAL_DIR = "../ARC-AGI-2/data/evaluation"

async def run_random_eval():
    # Get all tasks
    all_tasks = sorted(glob.glob(os.path.join(EVAL_DIR, "*.json")))
    
    # Random sample
    random.seed()  # Use current time for randomness
    selected = random.sample(all_tasks, min(NUM_TASKS, len(all_tasks)))
    
    print("=" * 60)
    print(f"🎲 RANDOM EVALUATION ({len(selected)} tasks)")
    print(f"   Available: {len(all_tasks)} tasks")
    print("=" * 60)
    print(f"\n📋 Selected: {[os.path.basename(p) for p in selected[:5]]}...")
    
    start_time = datetime.now()
    
    # Run tasks
    results = await run_tasks(selected, verbose=True)
    
    duration = (datetime.now() - start_time).total_seconds()
    
    # Summary using official ARC scoring
    total_tests = sum(r.n_test_cases for r in results)
    total_correct = sum(r.n_correct for r in results)
    tasks_fully_solved = sum(1 for r in results if r.score == 1.0)
    errors = sum(1 for r in results if r.error)
    avg_score = sum(r.score for r in results) / len(results) if results else 0
    
    print("\n" + "=" * 60)
    print("🏆 RESULTS (Official ARC Scoring)")
    print("=" * 60)
    print(f"📋 Tasks evaluated: {len(results)}")
    print(f"📋 Total test cases: {total_tests}")
    print(f"✅ Test cases correct: {total_correct}/{total_tests} ({100*total_correct/total_tests:.1f}%)")
    print(f"🎯 Tasks fully solved: {tasks_fully_solved}/{len(results)}")
    print(f"📊 Average task score: {avg_score*100:.1f}%")
    print(f"❌ Errors: {errors}")
    print(f"⏱️  Total time: {duration:.0f}s ({duration/len(results):.1f}s per task avg)")
    
    print("\n📋 Individual results:")
    for r in results:
        status = "✓" if r.score == 1.0 else "◐" if r.score > 0 else "✗"
        tests_info = f" ({r.n_correct}/{r.n_test_cases} tests)" if r.n_test_cases > 1 else ""
        print(f"   {status} {r.task_id}: {r.score*100:.0f}%{tests_info}")
    
    return results

# Run it!
results = asyncio.get_event_loop().run_until_complete(run_random_eval())


🎲 RANDOM EVALUATION (30 tasks)
   Available: 120 tasks

📋 Selected: ['7491f3cf.json', '9aaea919.json', 'b99e7126.json', '3a25b0d8.json', '45a5af55.json']...
🚀 Running 30 tasks (max 100 concurrent)

🚀 [1/30] Starting: 7491f3cf.json
🎭 ARC SOLVER
   Training examples: 4
   Test inputs: 1
------------------------------------------------------------
  👁️ Perceiving grids...

🚀 [2/30] Starting: 9aaea919.json
🎭 ARC SOLVER
   Training examples: 3
   Test inputs: 1
------------------------------------------------------------
  👁️ Perceiving grids...

🚀 [3/30] Starting: b99e7126.json
🎭 ARC SOLVER
   Training examples: 3
   Test inputs: 1
------------------------------------------------------------
  👁️ Perceiving grids...

🚀 [4/30] Starting: 3a25b0d8.json
   📋 Task has 2 test inputs
🎭 ARC SOLVER
   Training examples: 2
   Test inputs: 2
------------------------------------------------------------
  👁️ Perceiving grids...

🚀 [5/30] Starting: 45a5af55.json
🎭 ARC SOLVER
   Training examples: 2
   T

KeyboardInterrupt: 

    🔍 Self-verify: ✓ CORRECT (score=93)
     [gemini-flash] ✅ Self-verify=CORRECT (score=93)
  🔥 [gemini-flash] Self-verified solution! (score=93)
  🎯 Got 2 high-confidence (90+) solutions!
  ✓ Finished with 2 self-verified solution(s) (2 high-confidence 90+)
  ✓ Using 2 self-verified solution(s) (2 with score 90+)
------------------------------------------------------------
  📊 Final: 2 candidate(s), 1 test output(s)
  ✅ Source: self_verified
     1. gemini-flash: score=93, sv=CORRECT
     2. gemini-pro: score=93, sv=CORRECT

  📊 Result: ✅ CORRECT
     [gemini-flash] Attempt 1/10: ✓ Passed training
     [gemini-flash] Applied transform to 2 test inputs
     [gemini-flash] 🔍 Self-verification with gpt-5.2 (2 test(s))...
     [gemini-flash] Attempt 1/10: ✗ Error - Error code: 401 - {'error': {'message': 'User not 
     [gemini-flash] Attempt 2/10: ✗ Error - Error code: 401 - {'error': {'message': 'User not 
     [gemini-flash] Attempt 1/10: ✓ Passed training
     [gemini-flash] Applied 

In [ ]:
# ============================================================================
# RUN N RANDOM TASKS FROM FAILED LIST
# ============================================================================

import asyncio

# 👇 CONFIG
NUM_TASKS = 5  # Number of failed tasks to retry
FAIL_TASKS_FILE = "../fail_tasks.txt"
EVAL_DIR = "../ARC-AGI-2/data/evaluation"

async def run_failed_tasks():
    # Read failed tasks from file
    with open(FAIL_TASKS_FILE, 'r') as f:
        failed_ids = [line.strip() for line in f if line.strip()]
    
    # Build paths (add .json if needed)
    all_failed = []
    for task_id in failed_ids:
        if not task_id.endswith('.json'):
            task_id = task_id + '.json'
        path = os.path.join(EVAL_DIR, task_id)
        if os.path.exists(path):
            all_failed.append(path)
    
    print(f"✓ Found {len(all_failed)}/{len(failed_ids)} valid tasks in fail_tasks.txt")
    
    if not all_failed:
        print("❌ No valid failed tasks found!")
        return []
    
    # Random sample
    random.seed()  # Use current time for randomness
    selected = random.sample(all_failed, min(NUM_TASKS, len(all_failed)))
    
    print("=" * 60)
    print(f"🔄 RETRYING FAILED TASKS ({len(selected)} tasks)")
    print(f"   Total failed: {len(all_failed)} tasks")
    print("=" * 60)
    print(f"\n📋 Selected: {[os.path.basename(p) for p in selected[:5]]}...")
    
    start_time = datetime.now()
    
    # Run tasks
    results = await run_tasks(selected, verbose=True)
    
    duration = (datetime.now() - start_time).total_seconds()
    
    # Summary using official ARC scoring
    total_tests = sum(r.n_test_cases for r in results)
    total_correct = sum(r.n_correct for r in results)
    tasks_fully_solved = sum(1 for r in results if r.score == 1.0)
    errors = sum(1 for r in results if r.error)
    avg_score = sum(r.score for r in results) / len(results) if results else 0
    
    print("\n" + "=" * 60)
    print("🏆 RETRY RESULTS (Official ARC Scoring)")
    print("=" * 60)
    print(f"📋 Tasks evaluated: {len(results)}")
    print(f"📋 Total test cases: {total_tests}")
    print(f"✅ Test cases correct: {total_correct}/{total_tests} ({100*total_correct/total_tests:.1f}%)")
    print(f"🎯 Tasks fully solved: {tasks_fully_solved}/{len(results)}")
    print(f"📊 Average task score: {avg_score*100:.1f}%")
    print(f"❌ Errors: {errors}")
    print(f"⏱️  Total time: {duration:.0f}s ({duration/len(results):.1f}s per task avg)")
    
    print("\n📋 Individual results:")
    for r in results:
        status = "✓" if r.score == 1.0 else "◐" if r.score > 0 else "✗"
        tests_info = f" ({r.n_correct}/{r.n_test_cases} tests)" if r.n_test_cases > 1 else ""
        print(f"   {status} {r.task_id}: {r.score*100:.0f}%{tests_info}")
    
    # Show which previously failed tasks now pass
    newly_passing = [r.task_id for r in results if r.score == 1.0]
    if newly_passing:
        print(f"\n🎉 Fully solved: {newly_passing}")
    
    return results

# Run it!
results = asyncio.get_event_loop().run_until_complete(run_failed_tasks())
